<a href="https://colab.research.google.com/github/fcolombo7/AN2DL-2020/blob/main/Final%20Notebooks/3_TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AN2DL** - Image classification challenge

* **Colombo Filippo** - 10559531
* **Del Vecchio** Giovanni - 10570682




## Model with transfer learning



Configuration and constants:

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
#Random seed to make experiments reproducible
SEED = 1234
tf.random.set_seed(SEED)  

#Parameters
IMG_H, IMG_W = (400, 400)
BS = 32 #BATCH SIZE
VALIDATION_SPLIT = 0.2
DATA_AUGMENTATION = True

Load Google Drive to get the data and save the results:

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

cwd = os.getcwd()
drive_root_folder = '/content/drive/My Drive/ANN_project/'

Mounted at /content/drive


### Import the Mask Dataset

Check if the dataset has been already processed, otherwise unzip it.

**N.B.**: we are unzipping in the `/content` folder and not in `drive`.  
If you want to unzip in your drive folder, write `os.chdir(drive_root_folder)`



In [ ]:
#check if the dataset is already available
if not os.path.exists(cwd+'/MaskDataset'):
  !unzip '/content/drive/My Drive/ANN_project/artificial-neural-networks-and-deep-learning-2020.zip'
else:
  print('MaskDataset already loaded')

### Split the dataset

Definition of the `ImageDataGenerator` for _data augmentation_:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

if DATA_AUGMENTATION:
    train_data_gen = ImageDataGenerator(rotation_range=20, 
                                        width_shift_range=0.3, 
                                        height_shift_range=0.3, 
                                        zoom_range=0.4, 
                                        horizontal_flip=True, #
                                        #brightness_range = [0.6, 1.5], 
                                        shear_range=10, 
                                        channel_shift_range=100, 
                                        fill_mode='reflect', 
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)
    
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
import json

dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir = os.path.join(dataset_dir, "training")
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)
  dataframe = pd.DataFrame(dic.items())
  dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
  dataframe = dataframe.sample(frac=1, random_state=SEED)
  
  tot_length = dataframe.shape[0]
  valid = dataframe.iloc[:int(np.ceil(tot_length * VALIDATION_SPLIT)),:] 
  train = dataframe.iloc[int(np.ceil(tot_length * VALIDATION_SPLIT)):,:]
  train["class"] = train["class"].astype('string')
  valid["class"] = valid["class"].astype('string')
  
  train_gen = train_data_gen.flow_from_dataframe(train,
                                               training_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)
  
  validation_gen = valid_data_gen.flow_from_dataframe(valid,
                                               training_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

Found 4491 validated image filenames belonging to 3 classes.
Found 1123 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


### Build the Model

#### Create the dataset objects

In [ ]:
num_classes = len(train_gen.class_indices)
num_classes
train_ds = tf.data.Dataset.from_generator(lambda: train_gen, 
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes= ([None, IMG_H, IMG_W, 3], [None, num_classes]))

train_ds = train_ds.repeat()

valid_ds = tf.data.Dataset.from_generator(lambda: validation_gen,
                                          output_types = (tf.float32, tf.float32),
                                          output_shapes = ([None, IMG_H, IMG_W, 3], [None, num_classes]))
valid_ds = valid_ds.repeat()

####Use a pretrainde network as strating point: **VGG16**

In [ ]:
base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(IMG_H,IMG_W,3))

58892288/58889256 [==============================] - 3s 0us/step


In [ ]:
base_model.trainable = False
      
model = tf.keras.Sequential()
model.add(base_model)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=448, activation='relu'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 12, 12, 512)       14714688  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
dense (Dense)                (None, 448)               33030592  
_________________________________________________________________
dropout (Dropout)            (None, 448)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1347      
Total params: 47,746,627
Trainable params: 33,031,939
Non-trainable params: 14,714,688
_________________________________________________________________


### Compile the model

In [ ]:
def get_optimizer(lr_schedule = 1e-3):
  return tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [ ]:
def get_callbacks(name, es=True, patience = 10):
  obj = []

  exps_dir = os.path.join(drive_root_folder,'classification_result')
  if not os.path.exists(exps_dir):
    os.mkdir(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')
  temp_dir = name +'_'+ now

  #Model Checkpoints
  ckpt_dir = os.path.join(exps_dir, temp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

  obj.append(tf.keras.callbacks.ModelCheckpoint(filepath = os.path.join(ckpt_dir, 'cp.ckpt'),
                                                save_weights_only = True,
                                                save_best_only = True))
  
  #Tensor board logs
  tb_dir = os.path.join(exps_dir, temp_dir + '/tb_logs')
  if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
  obj.append(tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                            profile_batch = 0,
                                            histogram_freq = 1))
  if es:
    obj.append(tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = patience))

  return obj

In [ ]:
name = 'VGG-U448-DO01'
model.compile(optimizer=get_optimizer(),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x=train_ds,
          epochs=40,
          steps_per_epoch=len(train_gen),
          validation_data=valid_ds,
          validation_steps=len(validation_gen), 
          callbacks=get_callbacks(name),
          verbose = 1)

### Load the best model according to the checkpoints

In [ ]:
model_to_load = 'VGG-U448-DO01' #check the model name in the drive folder (the time stamp changes)
ckpt_path = os.path.join(drive_root_folder,'classification_result',model_to_load, 'ckpts','cp.ckpt')
model.load_weights(ckpt_path)

### Fine-tuning of the entire model
Unfreeze the base model and train the entire model end-to-end with a low learning rate.


In [ ]:
base_model.trainable = True
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 12, 12, 512)       14714688  
_________________________________________________________________
flatten (Flatten)            (None, 73728)             0         
_________________________________________________________________
dense (Dense)                (None, 448)               33030592  
_________________________________________________________________
dropout (Dropout)            (None, 448)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1347      
Total params: 47,746,627
Trainable params: 47,746,627
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

epochs = 20
model.fit(train_ds,
          epochs=epochs,
          validation_data=valid_ds,
          steps_per_epoch = len(train_gen),
          validation_steps = len(validation_gen),
          callbacks=get_callbacks('FT-'+model_to_laod, es = True, patience=8))

## Predictions


First load the test images:

In [ ]:
import pathlib
path = os.path.join(os.getcwd(),'MaskDataset', 'test')
data_dir = pathlib.Path(path)

test_image_filenames = list(data_dir.glob('*.jpg'))
len(test_image_filenames)

In [ ]:
test_normalization = tf.keras.Sequential([
                                       tf.keras.layers.experimental.preprocessing.Rescaling(1./255)                 
])

In [ ]:
def make_prediction():
  results = {}
  results_expanded = {}
  for test_image in test_image_filenames:
    img = tf.keras.preprocessing.image.load_img(test_image, target_size=(IMG_H, IMG_W))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    normalized_img = test_normalization(img_array, training=True)
    predictions = model.predict(normalized_img)
    results_expanded[os.path.basename(test_image)]=predictions
    results[os.path.basename(test_image)]=np.argmax(predictions)
  return results, results_expanded

In [ ]:
def create_csv(results, results_dir):
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:
        f.write('Id,Category\n')
        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
create_csv(make_prediction()[0], drive_root_folder)